<a href="https://colab.research.google.com/github/vasanthrohith/llm_finetune/blob/main/finetune_mitral_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

In [ ]:
from datasets import load_dataset

#dataset_name = "timdettmers/openassistant-guanaco" ###Human ,.,,,,,, ###Assistant

dataset_name = 'AlexanderDoria/novel17_test' #french novels
dataset = load_dataset(dataset_name, split="train")

In [ ]:
!pip install transformers trl accelerate torch bitsandbytes peft datasets -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 784.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
instruct_tune_dataset = load_dataset("mosaicml/instruct-v3")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/56167 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6807 [00:00<?, ? examples/s]

In [ ]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 56167
    })
    test: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 6807
    })
})

In [ ]:
instruct_tune_dataset = instruct_tune_dataset.filter(lambda x: x["source"] == "dolly_hhrlhf")

Filter:   0%|          | 0/56167 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6807 [00:00<?, ? examples/s]

In [ ]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 34333
    })
    test: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 4771
    })
})

In [ ]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(1000))

instruct_tune_dataset["test"] = instruct_tune_dataset["test"].select(range(200))

In [ ]:
instruct_tune_dataset["train"]

Dataset({
    features: ['prompt', 'response', 'source'],
    num_rows: 1000
})

In [ ]:
instruct_tune_dataset["test"]

Dataset({
    features: ['prompt', 'response', 'source'],
    num_rows: 200
})

In [ ]:
import pandas as pd

df_train = pd.read_csv('/content/train_dataset.csv')
df_test = pd.read_csv('/content/test_dataset.csv')


In [ ]:
dataset_train = load_dataset('csv', data_files='/content/train_dataset.csv')
dataset_test = load_dataset('csv', data_files='/content/test_dataset.csv')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset_train['train'] = dataset_train['train'].select(range(1000))
dataset_test['test'] = dataset_test['train'].select(range(200))

In [ ]:
dataset_train['train']

Dataset({
    features: ['Question', 'Answer'],
    num_rows: 1000
})

In [ ]:
print(len(df_train))
print(len(df_test))

1000
216


In [ ]:
def my_create_prompt(sample):
    bos_token = "<s>"
    system_message = "With the given question make sure you are giving the right answers related to Job Aid for Patient Navigators & Care Navigators for Optin/Opt-out & Data Rights Request for Novartis Patient Support Center"
    input = sample["Question"]
    response = sample["Answer"].replace("|||","\n")
    eos_token = "</s>"


    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "### Instruction:"
    full_prompt += "\n" + system_message
    full_prompt += "\n\n### Input:"
    full_prompt += "\n" + input
    full_prompt += "\n\n### Response:"
    full_prompt += "\n\n### Response:"
    full_prompt += "\n" + response
    full_prompt += eos_token



    return full_prompt

In [ ]:
my_create_prompt(sample=df_train.iloc[1])

'<s>### Instruction:\nWith the given question make sure you are giving the right answers related to Job Aid for Patient Navigators & Care Navigators for Optin/Opt-out & Data Rights Request for Novartis Patient Support Center\n\n### Input:\nCould you provide information on the Training Requirements mentioned?\n\n### Response:\n\n### Response:\nThe training requirements mentioned in the context are as follows:\n- Fabhalta Patient Navigators: Read & Understand\n - Fabhalta PSC Operations (Director, AD & TLs): Read & Understand\n - NPS ERC: Read & Understand\n - NPS PE Analysts for Fabhalta: Read & Understand\n - NPS DST for Fabhalta: Read & Understand\n - NPS QA: Read & Understand\n\n It is important to note that if a NPS role not listed in the above training grid assists with monitoring in-person welcome visits, they will need to complete the required training.</s>'

In [ ]:
def create_prompt(sample):
    """
    Update the prompt template:
    Combine both the prompt and input into a single column.
    """
    bos_token = "<s>"
    original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    system_message = "Use the provided input to create an instruction that could have been used to generate the response with an LLM."
    response = sample["prompt"].replace(original_system_message, "").replace("\n\n### Instruction\n", "").replace("\n### Response\n", "").strip()
    input = sample["response"]
    eos_token = "</s>"

    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "### Instruction:"
    full_prompt += "\n" + system_message
    full_prompt += "\n\n### Input:"
    full_prompt += "\n" + input
    full_prompt += "\n\n### Response:"
    full_prompt += "\n" + response
    full_prompt += eos_token

    return full_prompt

In [ ]:
create_prompt(instruct_tune_dataset["train"][0])

'<s>### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM.\n\n### Input:\nThere are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.\n\n### Response:\nWhat are different types of grass?</s>'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
def generate_response(prompt, model):
    encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')

    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0].replace(prompt, "")

In [ ]:
prompt="### Instruction:\nWith the given question make sure you are giving the right answers related to Job Aid for Patient Navigators & Care Navigators for Optin/Opt-out & Data Rights Request for Novartis Patient Support Center### Input:\nEnumerate what is mentioned in the review cycle\n\n### Response:"




In [ ]:
generate_response(prompt, model)

'<s> \nThe review cycle for the Job Aid for Patient Navigators & Care Navigators for Optin/Opt-out & Data Rights Request for Novartis Patient Support Center includes:\n\n1. Drafting the job aid\n2. Review by Subject Matter Experts (SMEs)\n3. Approval by Legal Counsel\n4. Review by Quality Assurance (QA) Team\n5. Final approval by the Novartis Patient Support Center Leadership\n6. Distribution to applicable teams and training of Patient Navigators & Care Navigators\n7. Ongoing maintenance and updates as needed.</s>'

In [ ]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "mistral_instruct_generation",
  #num_train_epochs=5,
  max_steps = 100, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 4,  #from 4
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=20, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  bf16=True,  #from bf16=True
  lr_scheduler_type='constant',
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=my_create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=args,
  train_dataset=dataset_train['train'],
  eval_dataset=dataset_test['test']
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:341: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
20,1.929700,1.516923
40,1.371600,1.291100
60,1.169600,1.192720
80,1.085700,1.148212
100,1.040200,1.108730


TrainOutput(global_step=100, training_loss=1.3830864906311036, metrics={'train_runtime': 92.9326, 'train_samples_per_second': 2.152, 'train_steps_per_second': 1.076, 'total_flos': 2192769967718400.0, 'train_loss': 1.3830864906311036, 'epoch': 0.24})

In [ ]:
trainer.save_model("mistral_instruct_generation")

In [ ]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

In [ ]:
prompt="### Instruction:\nWith the given question make sure you are giving the right answers related to Job Aid for Patient Navigators & Care Navigators for Optin/Opt-out & Data Rights Request for Novartis Patient Support Center### Input:\nEnumerate what is mentioned in the review cycle\n\n### Response:"



In [ ]:
generate_response(prompt = prompt, model = merged_model)

"<s> ### Instruction:\nWith the given question make sure you are giving the right answers related to Job Aid for Patient Navigators & Care Navigators for Optin/Opt-out & Data Rights Request for Novartis Patient Support Center### Input:\nEnumerate what is mentioned in the review cycle\n\n### Response:\n\nThe following are the steps mentioned in the review cycle for Optin/Opt-out & Data Rights Request for Novartis Patient Support Center:\n\n1. Review and identify the patient's request.\n2. Determine if the request is a data rights request, opt-in/opt-out request, medical records request, or a miscellaneous request.\n3. Verify the validity of the request.\n4. Confirm if authorization is required for the request.\n5. Confirm that the request is complete and accurate.\n6. Gather all necessary information before responding to the request.\n7. Respond to the patient's request within the time limit.\n8. If the patient's request is not met, determine the reason for the denial.\n9. Update the re

In [ ]:
"The training requirements mentioned in the context are as follows:|||- Fabhalta Patient Navigators: Read & Understand||| - Fabhalta PSC Operations (Director, AD & TLs): Read & Understand||| - NPS ERC: Read & Understand||| - NPS PE Analysts for Fabhalta: Read & Understand||| - NPS DST for Fabhalta: Read & Understand||| - NPS QA: Read & Understand|||||| It is important to note that if a NPS role not listed in the above training grid assists with monitoring in-person welcome visits, they will need to complete the required training."